<a href="https://colab.research.google.com/github/sdlt2003/lain-bootleg-bootleg-to-NDS/blob/master/HB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solucionario Examen Heurísticos

Este cuaderno contiene las implementaciones completas y correctas de los tres algoritmos clave: PSO, Dominancia de Pareto y ACO.

---

## 1. Particle Swarm Optimization (PSO)

### Fórmulas Implementadas
1. **Velocidad:** $v_{t+1} = w \cdot v_t + c_1 r_1 (p_{best} - x_t) + c_2 r_2 (g_{best} - x_t)$
2. **Posición:** $x_{t+1} = x_t + v_{t+1}$

In [ ]:
import numpy as np

class Particle:
    def __init__(self, bounds):
        self.position = np.random.uniform(bounds[0], bounds[1], size=len(bounds))
        self.velocity = np.random.uniform(-1, 1, size=len(bounds))
        self.p_best_position = self.position.copy()
        self.p_best_value = float('inf')

    def update_velocity(self, g_best_position, w, c1, c2):
        """Actualización completa de velocidad con vectores aleatorios."""
        dim = len(self.position)

        # 1. Vectores aleatorios (estocasticidad independiente por dimensión)
        r1 = np.random.uniform(0, 1, size=dim)
        r2 = np.random.uniform(0, 1, size=dim)

        # 2. Cálculo de componentes
        inertia = w * self.velocity
        cognitive = c1 * r1 * (self.p_best_position - self.position)
        social = c2 * r2 * (g_best_position - self.position)

        # 3. Actualización
        self.velocity = inertia + cognitive + social

    def update_position(self):
        """Actualización simple de posición."""
        self.position = self.position + self.velocity

# --- TEST RÁPIDO PSO ---
def run_pso_test():
    print("--- Testeando Solución PSO ---")
    p = Particle([-10, 10])
    g_best = np.array([0, 0])

    print(f"Velocidad Inicial: {p.velocity}")
    p.update_velocity(g_best, w=0.5, c1=1.5, c2=1.5)
    print(f"Velocidad Actualizada: {p.velocity}")

    pos_old = p.position.copy()
    p.update_position()
    print(f"Posición (t): {pos_old} -> (t+1): {p.position}")

run_pso_test()

---
## 2. Dominancia y Frontera de Pareto

### Definición de Dominancia (Minimización)
Una solución $A$ domina a $B$ ($A \prec B$) si:
1. $\forall i, f_i(A) \leq f_i(B)$ (A no es peor que B en nada)
2. $\exists k, f_k(A) < f_k(B)$ (A es estrictamente mejor que B en algo)

In [ ]:
def dominates(sol_a, sol_b):
    """
    Devuelve True si sol_a domina a sol_b (Minimización).
    sol_a y sol_b son tuplas/listas de objetivos.
    """
    # Condición 1: A es mejor o igual en todo
    # Usamos zip para comparar elemento a elemento
    condition_1 = all(a <= b for a, b in zip(sol_a, sol_b))

    # Condición 2: A es estrictamente mejor en al menos uno
    condition_2 = any(a < b for a, b in zip(sol_a, sol_b))

    return condition_1 and condition_2

def get_pareto_front(solutions):
    """Filtra las soluciones no dominadas."""
    pareto_front = []

    for i, sol_a in enumerate(solutions):
        is_dominated = False
        for j, sol_b in enumerate(solutions):
            if i == j: continue

            # Si alguien (B) domina a la actual (A), A no es frontera
            if dominates(sol_b, sol_a):
                is_dominated = True
                break

        if not is_dominated:
            pareto_front.append(sol_a)

    return pareto_front

# --- TEST RÁPIDO PARETO ---
def run_pareto_test():
    print("\n--- Testeando Solución Pareto ---")
    # (Coste, Tiempo) - Minimizar ambos
    data = [(2, 10), (4, 6), (8, 2), (6, 8), (5, 5)]
    # Esperado: (2,10), (4,6), (5,5), (8,2).
    # (6,8) es dominada por (4,6) y (5,5).

    front = get_pareto_front(data)
    print(f"Soluciones originales: {data}")
    print(f"Frontera de Pareto: {front}")

run_pareto_test()

---
## 3. Ant Colony Optimization (ACO)

### Lógica Implementada
1. **Selección Probabilística:** $P_{ij} = \frac{\tau_{ij}^\alpha \cdot \eta_{ij}^\beta}{\sum (...)}$
2. **Actualización Feromonas:** $\tau_{ij} \leftarrow (1-\rho)\tau_{ij} + \sum \frac{1}{Coste}$

In [ ]:
class AntColonySolver:
    def __init__(self, distance_matrix, alpha=1, beta=2, rho=0.5):
        self.dist_matrix = distance_matrix
        self.n_cities = len(distance_matrix)
        self.alpha = alpha
        self.beta = beta
        self.rho = rho

        self.pheromone = np.ones((self.n_cities, self.n_cities)) / self.n_cities
        with np.errstate(divide='ignore'):
            self.heuristic = 1.0 / self.dist_matrix
        np.fill_diagonal(self.heuristic, 0)

    def select_next_node(self, current_node, visited_mask):
        """Selecciona el siguiente nodo usando la regla de transición."""
        candidates = np.where(~visited_mask)[0]
        if len(candidates) == 0: return None

        # 1. Obtener valores de matrices globales
        tau = self.pheromone[current_node, candidates]
        eta = self.heuristic[current_node, candidates]

        # 2. Calcular numerador de probabilidad
        numerators = np.power(tau, self.alpha) * np.power(eta, self.beta)

        # 3. Calcular denominador y probabilidades finales
        denominator = np.sum(numerators)
        if denominator == 0:
            probs = np.ones(len(candidates)) / len(candidates)
        else:
            probs = numerators / denominator

        # 4. Selección por Ruleta
        return np.random.choice(candidates, p=probs)

    def update_pheromones(self, all_paths, all_costs):
        """Evaporación y depósito de feromonas."""
        # A. Evaporación global
        self.pheromone = self.pheromone * (1 - self.rho)

        # B. Depósito basado en calidad de solución
        for path, cost in zip(all_paths, all_costs):
            deposit = 1.0 / cost

            # Depositar en cada arista del camino (ida y vuelta)
            for i in range(len(path) - 1):
                u, v = path[i], path[i+1]
                self.pheromone[u, v] += deposit
                self.pheromone[v, u] += deposit

            # Cerrar el ciclo
            u, v = path[-1], path[0]
            self.pheromone[u, v] += deposit
            self.pheromone[v, u] += deposit

# --- TEST RÁPIDO ACO ---
def test_aco_logic():
    print("\n--- Testeando Solución ACO ---")
    dists = np.array([[0, 2, 10], [2, 0, 5], [10, 5, 0]], dtype=float)
    solver = AntColonySolver(dists, rho=0.5)

    # Simular actualización de feromona
    old_ph = solver.pheromone[0, 1]
    solver.update_pheromones([[0, 1, 2]], [7.0]) # Camino 0->1->2->0 (Coste 2+5+10=17)
    print(f"Feromona: {old_ph:.4f} -> {solver.pheromone[0, 1]:.4f} (Debería cambiar)")

test_aco_logic()

Para justificar los resultados de una metaheurística en un examen o informe técnico, no basta con decir "ha dado un número bajo". Debes analizar la dinámica del algoritmo.

Los profesores buscan que identifiques el **equilibrio entre Exploración (Exploration) y Explotación (Exploitation)**.

Aquí tienes una guía técnica de cómo justificar los resultados basándote en tres escenarios comunes:

### 1. Escenario Ideal: Convergencia Rápida y Buen Resultado

El algoritmo encuentra una solución muy buena (cerca del óptimo conocido o lógico) en pocas iteraciones y la curva de error baja suavemente.

* **Qué decir:**
> "El algoritmo muestra un **compromiso (trade-off) adecuado entre exploración y explotación**. En las primeras fases, la diversidad poblacional permitió muestrear ampliamente el espacio de búsqueda. Posteriormente, la presión selectiva (ya sea por feromonas en ACO o atracción al líder en PSO) logró refinar la solución."


* **Justificación técnica:**
* **ACO:** "Los parámetros  y  están balanceados. La evaporación () fue suficiente para eliminar rastros subóptimos sin destruir la memoria del sistema prematuramente."
* **PSO:** "La inercia () se redujo adecuadamente o los coeficientes cognitivo () y social () permitieron que las partículas oscilaran alrededor del óptimo antes de converger."



### 2. Escenario Malo: Convergencia Prematura (Premature Convergence)

El algoritmo se detiene rápido, todas las soluciones son iguales (varianza 0), pero el resultado es malo (un óptimo local).

* **Qué decir:**
> "Se observa un fenómeno de **estancamiento en óptimos locales**. El algoritmo perdió diversidad poblacional demasiado rápido, favoreciendo la explotación excesiva antes de explorar regiones prometedoras del espacio de búsqueda."


* **Justificación técnica (Causas probables):**
* **ACO:** "Probablemente la evaporación () es demasiado baja (la feromona se acumula muy rápido) o el peso de la feromona () es excesivo frente a la heurística (). Esto causa un ciclo de retroalimentación positiva demasiado fuerte."
* **PSO:** "El coeficiente social () es demasiado alto respecto al cognitivo o la inercia, provocando que todo el enjambre colapse hacia la posición del líder actual () sin explorar alternativas."



### 3. Escenario Malo: Convergencia Lenta o Divergencia

El algoritmo sigue iterando, las soluciones cambian mucho, pero no mejoran o mejoran muy lentamente.

* **Qué decir:**
> "El algoritmo sufre de **exceso de exploración (random walk)**. No existe suficiente presión de selección para guiar la búsqueda hacia zonas prometedoras. El sistema se comporta de manera casi aleatoria."


* **Justificación técnica:**
* **ACO:** "La tasa de evaporación es demasiado alta (el sistema 'olvida' demasiado rápido) o el factor  es insignificante."
* **PSO:** "La velocidad de las partículas es excesiva (falta de constricción o inercia demasiado alta), impidiendo que se 'asienten' en los mínimos encontrados."



---

### Ejemplo Práctico de Texto para tu Examen

Supongamos que ejecutas el **ACO para el TSP de España** que te pasé y obtienes una ruta de 3500km (que es razonable pero no perfecta).

**Ejemplo de Justificación (Nivel Examen):**

> "La ejecución del algoritmo ACO sobre las 48 capitales resultó en una distancia de 3500km.
> **Análisis de Convergencia:** La curva de aptitud (fitness) muestra un descenso rápido en las primeras 20 iteraciones, estabilizándose (asíntota) a partir de la iteración 60. Esto indica que el algoritmo es capaz de identificar rápidamente la estructura global del problema (el perímetro de la península).
> **Estabilidad:** Al ejecutar el algoritmo 10 veces, la desviación estándar de los resultados es baja (), lo que sugiere que la configuración de parámetros () es **robusta**.
> **Crítica:** Sin embargo, la solución final presenta algunos cruces de aristas locales. Esto sugiere que, aunque la **intensificación** (explotación) es correcta, podría beneficiarse de un mecanismo de búsqueda local adicional (como 2-opt) post-ACO para refinar esos óptimos locales, ya que ACO por sí solo tiene dificultades para el 'ajuste fino' final en espacios discretos."

### Vocabulario Clave para usar (Cheat Sheet)

Úsalo para sonar técnico y preciso:

1. **Landscape (Paisaje de búsqueda):** "El paisaje presenta múltiples valles (óptimos locales)."
2. **Stagnation (Estancamiento):** "El algoritmo se estancó en la iteración 50."
3. **Diversity (Diversidad):** "La pérdida de diversidad en el enjambre..."
4. **Selection Pressure (Presión selectiva):** "Aumentar  incrementa la presión selectiva hacia nodos cercanos."
5. **Robustness (Robustez):** "El algoritmo es robusto ante cambios en la semilla aleatoria."